The aim: To use estimated parameter inputs in a sampling technique to outpute a synthetic time series dataset

Method 1: To sample from a probability distribution and generate synthetic time series data based on estimates or intial guesses of the parameters.

# Bayesian Inference with MCMC 

Explanation of method: Bayesian inference is a pwoerful framework that allows you to estimate probability distributions of parameters given observed data. By combining Bayesian inference with MCMC sampling techniques, you can sample from the posterior distribution of the parameters and generate synthetic time series data that reflects the uncertainty in the parameter estimates.This involves specifying prior distributions for the parameters, formulating the likelihood function based on the synthetic data and using MCMC method to sample from the join posterior distribution of the parameters.

Pseudocode / breakdown of the stages involved in generating a time series dataset using Bayesian inference with MCMC

### Step 1: Define the model 
Specify the mathematical model that describes the time series data generation process. This includes equations, function or algorithms that related the parameters to the time series data.

### Step 2: Define the Priors
Specify the prior distribution for the parameters. Priors represet your beliefs or knowlege about the parameter values before observing the data. You can choose appropriate probability distributions based on your understanding of the problem

### Step 3: Define the likelihood
Formulate the likelihood function that describes the probability of observing the data given the prameters. The likelihood represents the statistical relationship between the parameters and the observed time series data.

### Step 4: Define the Posterior
Calculate the posterior distribution of the parameters using Bayes Theorem. The posterior is proportional to the product of the prior and the likelihood.

### Step 5: MCMC Sampling 
Implement an MCMC sampling algorithm. 2 possibilities are Metropolis-Hastings algo or the Hamiltonian Monte Carlo algo. These MCMC sampling algos are to sample from the posterior distribution. MCMC lets you explore the parameter space and generate a chain of samples that are representive of the posterior distribution.

### Step 6: Burn-in and Thinning

Discard an intial portion of the MCMC samples to remove any dependence on the inital parameter values (burn-in). 
Then, thin out the remaining samples to reduce auto-colleration between successive samples and improves computational efficiency

### Step 7: Generate synthetic dataset

Use the retained MCMC samples to generate synthetic time series datasets. For each sample of the parameters, apply the model equations or algorithms to produce a corresponding synthetic time series dataset.

In [1]:
%%capture

!pip install pymc3

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pymc3 as pm

WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


Step 1: Define the model

\begin{align}
f(t, y) &= \gamma + K\left(e \cos(\omega) + \cos{\left(\phi(t) + \omega \right)}\right),
\quad \phi = 2 {\rm arc} \tan \left( \sqrt{\frac{1+e}{1 - e}}\tan \frac{E(t)}{2}\right), 
\quad E(t) = e \sin(E(t)) + 2 \pi \frac{t - t_p}{P} 
\end{align}

In [6]:
'''
Step 1: Define the model
'''
def genModel(t, y): #basically f(t, y) equation
    gamma, K, e, omega, tp, P = y
    phi = simulatePhi(t, y)
    r1 = np.cos(phi + omega)
    r2 = e*np.cos(omega)
    f = gamma + K * (r1+r2)
    return f

def simulatePhi(t, y, steps = 10): #calculating numerical solutions to phi equation
    gamma, K, e, omega, tp, P = y
    s = tp
    eta = t / (steps - 1)
    E = 0
    for it in range(steps):
        s = s + eta
        g = np.sqrt((1 + e)/(1 - e))*np.tan(E)/2 #Dr Colombos version I think the tan includes the E(t)/2 inside the brackets.
        phi = 2 * np.arctan(g)  
        E = e*np.sin(E) + 2*np.pi/P * (s - tp)
    return phi

Step 2 -- Step 6
 Define priors, Define Likelihood, Define the posterior, MCMC Sampling, Burn-in and Thinning

In [10]:
import pymc3 as pm
def generateData(params, t):
    with pm.Model() as model:
        # Priors
        gamma = pm.Normal('gamma', mu=params['gamma_mu'], sd=params['gamma_sd'])
        K = pm.Normal('K', mu=params['K_mu'], sd=params['K_sd'])
        e = pm.Uniform('e', lower=params['e_lower'], upper=params['e_upper'])
        omega = pm.Normal('omega', mu=params['omega_mu'], sd=params['omega_sd'])
        tp = pm.Normal('tp', mu=params['tp_mu'], sd=params['tp_sd'])
        P = pm.Normal('P', mu=params['P_mu'], sd=params['P_sd'])
        
        #generate true trajectory
        trueTrajectory = genModel(t, [gamma, K, e, omega, tp, P])
        
        #generate observed data
        obsSTD = pm.HalfNormal('obs_sd', sd = params['obs_sd'])
        
        #likelihood 
        obs = pm.Normal('obs', mu = trueTrajectory, sd = obsSTD, observed = True)
        
        #perform MCMC sampling
        
        trace = pm.sample(params['n_samples'], tune=params['n_tune'], cores = params['n_cores'], random_seed = params['random_seed'], progressbar = False)
        
        #get the posterior distribution of the parameters
        
        posterior_samples = pm.trace_to_dataframe(trace, varnames = ['gamma', 'K', 'e', 'omega', 'tp', 'P'])
        
        #burn in and thinning
        posterior_samples = posterior_samples[params['n_burn_in']::params['thinning']]
        
    return trueTrajectory, posterior_samples

params = {
    'gamma_mu': 0.01,
    'gamma_sd': 0.01,
    'K_mu': 10,
    'K_sd': 1,
    'e_lower': 0,
    'e_upper': 1,
    'omega_mu': 1,
    'omega_sd': 0.1,
    'tp_mu': 100,
    'tp_sd': 10,
    'P_mu': 1000,
    'P_sd': 100,
    'obs_sd': 0.1,
    'n_samples': 1000,
    'n_tune': 1000,
    'n_cores': 1,
    'random_seed': 12345,
    'n_burn_in': 500,
    'thinning': 2
}


Step 7: Generate synthetic dataset

In [11]:
t = np.arange(0, 1000)
trueTrajectory, posterior_samples = generateData(params, t)


/home/abdullah/anaconda3/lib/python3.9/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
/home/abdullah/anaconda3/lib/python3.9/site-packages/theano/tensor/elemwise.py:826: RuntimeWarning: divide by zero encountered in log
  variables = ufunc(*ufunc_args, **ufunc_kwargs)
/home/abdullah/anaconda3/lib/python3.9/site-packages/theano/tensor/elemwise.py:826: RuntimeWarning: invalid value encountered in multiply
  variables = ufunc(*ufunc_args, **ufunc_kwargs)
/home/abdullah/anaconda3/lib/python3.9/site-packages/theano/tensor/elemwise.py:826: RuntimeWarning: invalid value encountered in log
  variables = ufunc(*ufunc_args, **ufunc_kwargs)
/home/abdullah/anaconda3/lib/python

ValueError: cannot reshape array of size 0 into shape (0,newaxis)

Plot data